In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import cc3d

import cv2
from PIL import Image

import pandas as pd

from tqdm import tqdm_notebook as tqdm

from skimage.measure import regionprops, label, find_contours
from scipy.spatial.distance import cdist

import scipy.ndimage as ndimage


ModuleNotFoundError: No module named 'matplotlib'

In [4]:
!conda install numpy==1.17.4

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - numpy==1.17.4

Current channels:

  - https://conda.anaconda.org/conda-forge/osx-arm64
  - https://conda.anaconda.org/conda-forge/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [ ]:
#値を-1から1に正規化する関数
def normalize_x(image):
    return image / 127.5 - 1


#値を0から1正規化する関数
def normalize_y(image):
    return image / 255


#値を0から255に戻す関数
def denormalize_y(image):
    return image * 255

In [ ]:
# インプット画像を読み込む関数
def load_X_gray(folder_path, size_x, size_y):
    
    image_files = []

    #image_files = os.listdir(folder_path)
       
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else :
            pass
        
    image_files.sort()
    print (image_files)
    #image_files = image_files[1:]
    images = np.zeros((len(image_files), size_x, size_y, 1), np.float32)
    for i, image_file in enumerate(image_files):
        image = cv2.imread(folder_path + os.sep + image_file, cv2.IMREAD_GRAYSCALE)
        print (image.shape)
        image = cv2.resize(image, (size_y, size_x))
        image = image[:, :, np.newaxis]
        images[i] = normalize_x(image)
    return images, image_files


# ラベル画像を読み込む関数
def load_Y_gray_(folder_path, size_x, size_y, thresh = None , normalize = True):
    image_files = []
    #image_files = os.listdir(folder_path)
    
    for file in os.listdir(folder_path):
        base , ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else:
            pass
        
    image_files.sort()
    print(image_files)
    images = np.zeros(
        (
            len(image_files) ,
            size_x ,
            size_y ,
            1
        ) ,
        np.float32
    )
    
    for i , image_file in enumerate(image_files):
        image = cv2.imread(
            folder_path + os.sep + image_file ,
            cv2.IMREAD_GRAYSCALE
        )
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = cv2.resize(image ,(size_y ,size_x))
        image = image[ : , : , np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
            
    print(images.shape)
    
    return images , image_files


# インプット画像を読み込む関数
def load_X_gray(folder_path):
    
    image_files = []

    #image_files = os.listdir(folder_path)
       
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else :
            pass
        
    image_files.sort()
    print (image_files)
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    #image_files = image_files[1:]
    images = np.zeros((len(image_files), img.shape[0], img.shape[1], 1), np.float32)
    for i, image_file in enumerate(image_files):
        image = cv2.imread(folder_path + os.sep + image_file, cv2.IMREAD_GRAYSCALE)
        print (image.shape)
        #image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        image = image[:, :, np.newaxis]
        images[i] = normalize_x(image)
    return images, image_files


# ラベル画像を読み込む関数
def load_Y_gray(folder_path, thresh = None , normalize = True):
    image_files = []
    #image_files = os.listdir(folder_path)
    
    for file in os.listdir(folder_path):
        base , ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else:
            pass
        
    image_files.sort()
    print(image_files)
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    images = np.zeros(
        (
            len(image_files) ,
            img.shape[0] ,
            img.shape[1] ,
            1
        ) ,
        np.float32
    )
    
    for i , image_file in enumerate(image_files):
        image = cv2.imread(
            folder_path + os.sep + image_file ,
            cv2.IMREAD_GRAYSCALE
        )
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = cv2.resize(image ,(img.shape[1] ,img.shape[0]))
        image = image[ : , : , np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
            
    print(images.shape)
    
    return images , image_files

In [ ]:
def Quantify_Mito_(imgs):
    S = 0
    for i in range(imgs.shape[0]):
        if np.max(imgs[i]) > 0:
            contours, hierarchy = cv2.findContours(imgs[i],
                                       cv2.RETR_TREE,
                                       cv2.CHAIN_APPROX_SIMPLE)
            img = np.zeros((imgs.shape[1], imgs.shape[2], 1), np.float32)
            line = cv2.drawContours(img, contours, -1, (1,0,0), 1)
        
            x = imgs.shape[1]
            y = imgs.shape[2]
        
            counter = line[:,:,0]
            #print(line.shape, counter.shape)
            counter = cv2.line(counter,(0,y-1),(x,y-1),(0,0,0),thickness=1)
            counter = cv2.line(counter,(0,1),(x,1),(0,0,0),1)
            counter = cv2.line(counter,(1,0),(x-1,0),(0,0,0),1)
            counter = cv2.line(counter,(1,y),(x-1,y),(0,0,0),1)
        
            #quantified_line = quantitative_gap(line[:,:,0])
            quantified_line = np.sum(line[:,:,0])
            S += quantified_line
    return S

In [ ]:
def pick_center(imgs):
    label_img = regionprops(imgs)
    for props in label_img:
        G = props.centroid
        
    return G

In [ ]:
def pick_counters(imgs):
    
    """
        imgs : ndarray(Z, Y, X, 1)
        
        return : list( array(Z, Y, X) )
    """
    
    counter_points = []
    
    for i in range(imgs.shape[0]):
        counters = find_contours(imgs[i], 1)
        if counters:
            for point in counters[0]:
                counter_points.append(
                    (i, int(point[0]), int(point[1]))
                )
                
    
    return counter_points

In [ ]:
def pick_cristae_counter(lamellar_images, tubular_images):
    
    """
        Args : 
            lamellar_images (numpy.ndarray) : Z, Y, X, 1
            tubular_images (numpy.ndarray) : Z, Y, X, 1
            
        Returns : 
            counter_images (numpy.ndarray) : Z, Y, X, 1
    """
    
    lst = []
    
    cristae_imgs = tubular_images * 255  + lamellar_images * 255
    cristae_imgs = np.where(cristae_imgs > 0, 1, 0).astype("uint8")
    
    for z in range(lamellar_images.shape[0]):
        cristae_counters, cristae_hierarchy = cv2.findContours(cristae_imgs[z], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cristae_counter_img = np.zeros((cristae_imgs.shape[1], cristae_imgs.shape[2], 1), np.float32)
        cristae_line = cv2.drawContours(cristae_counter_img, cristae_counters, -1, (1,0,0), 1)
        
        x = lamellar_images.shape[1]
        y = lamellar_images.shape[2]
        
        counter = cristae_line[:,:,0]
        counter = cv2.line(counter,(0,y-1),(x,y-1),(0,0, 0),1)
        counter = cv2.line(counter,(0,1),(x,1),(0,0, 0),1)
        counter = cv2.line(counter,(1,0),(x-1,0),(0,0, 0),1)
        counter = cv2.line(counter,(1,y),(x-1,y),(0,0, 0),1)
        
        lst.append(counter)
        
    return np.array(lst)

In [ ]:
def culcurate_cristae_SV(lamellar_images, tubular_images):
    
    """
        Args : 
            lamellar_images (numpy.ndarray) : Z, Y, X, 1
            tubular_images (numpy.ndarray) : Z, Y, X, 1
            
        Returns : 
            counter_images (numpy.ndarray) : Z, Y, X, 1
    """
    
    cristae_counter_img = pick_cristae_counter(lamellar_images, tubular_images)
    
    lamellar_conuters = np.count_nonzero(
        (cristae_counter_img > 0) & (lamellar_images > 0)
    )    
    
    tubular_counters = np.count_nonzero(
        (cristae_counter_img > 0) & (tubular_images > 0)
    )
    
    return {
        "lamellar_counters" : lamellar_conuters, 
        "tubular_counters" : tubular_counters
    }

In [ ]:
def culc_gravity(imgs):
    
    """
        imgs : ndarray (Z, Y, X, 1)
    """
    
    label_img = regionprops(imgs)
    for props in label_img:
        G = props.centroid
        
    return G

In [ ]:
def culc_axis(imgs, G):
    
    """
        imgs : ndarray (Z, Y, X, 1)
        G : ndarray(Z, Y, X)
    """
    
    counter_imgs = np.zeros((imgs.shape[0], imgs.shape[1], imgs.shape[2], 1), np.float32)

    counter_points = []
    for i in range(counter_imgs.shape[0]):
        counters = find_contours(imgs[i], 1)
        if counters:
            for point in counters[0]:
                counter_imgs[i][int(point[0]), int(point[1]), 0] = 255
                counter_points.append(
                    (i, int(point[0]), int(point[1]))
                )
                    
                    
    max_L = 0
    max_point = (0, 0, 0)
    min_point= (0, 0, 0)

    for counter_point in counter_points:
    
            if counter_point[0] < G[0]:
        
                '''
                      x        dx           Xg
                    ( y ) = ( dy ) n + ( Yg )
                      z        dz           Zg
              
                    dz = 1;
                    (Xg, Yg, Zg)は重心
                '''
        
                dz = G[0] - counter_point[0]
                dy = (G[1] - counter_point[1]) / dz
                dx = (G[2] - counter_point[2]) / dz
        
                max_counter_point = (counter_points[0])
                for nz in range(int(imgs.shape[0] - G[0])):
                    x_ = int(G[2] + dx * nz)
                    y_ = int(G[1] + dy * nz)
                    z_ = int(G[0] + nz)
            
                    if y_ < imgs.shape[1] and x_ < imgs.shape[2] and y_ > 0 and x_ > 0 and imgs[z_, y_, x_] > 0 :
                        max_counter_point = (z_, y_, x_)
                L = sum([ float(max_counter_point[i] - float(counter_point[i])) ** 2 for i in range(3) ])
        
                if L > max_L:
                    max_L = L
                    max_point = max_counter_point
                    min_point = counter_point
                    
    return max_L ** 0.5, max_point, min_point

In [ ]:
def rotate_x(X):
    r = (X[1] ** 2 + X[2] ** 2)**0.5
    C = X[2] / r
    S = X[1] / r
    
    return np.matrix((
        (1, 0, 0),
        (0, C, -S),
        (0, S, C)
    ))


def rotate_y(Y):
    r = (Y[0] ** 2 + Y[2] ** 2) ** 0.5
    C = Y[2] / r
    S = Y[0] / r
    
    return np.matrix((
        (C, 0, -S),
        (0, 1, 0),
        (S, 0, C)
    ))


def caluculate_rotate_matrix(edge_point, center):
    point = np.array((
        int(edge_point[2]) - int(center[2]),
        int(edge_point[1]) - int(center[1]),
        int(edge_point[0]) - int(center[0])
    ))
    
    first_rotated_matrix = rotate_x(point)
    
    first_rotated_point = np.array(np.dot(first_rotated_matrix, point), dtype = int)[0]
    

    second_rotated_matrix = rotate_y(first_rotated_point)
    
    print(np.array(np.dot(second_rotated_matrix, first_rotated_point), dtype=int)[0])
    
    return first_rotated_matrix, second_rotated_matrix


def move_target(point, center):
    return(np.array((
        point[0] - int(center[2]),
        point[1] - int(center[1]),
        point[2] - int(center[0])
    )))


def culc_short_axes(imgs, G, min_point):
    
    """
        imgs : ndarray (Z, Y, X, 1)
        G : ndarray(Z, Y, X)
        max_point : ndarray(Z, Y, X)
    """
    
    
    target_coordinate = (imgs == 255).nonzero()
    target_matrix = [ (x, y, z) for z, y, x in zip(target_coordinate[0], target_coordinate[1], target_coordinate[2]) ]
    
    first_matrix, second_matrix = caluculate_rotate_matrix(min_point, G)
    
    rotated_matrix = [
        np.array(np.dot(
            second_matrix, np.array(np.dot(first_matrix, move_target(target_point, G)), dtype=int)[0]
        ), dtype=int)[0] for target_point in target_matrix
    ]
    
    min_x = min(list( node[0] for node in rotated_matrix))
    max_x = max(list( node[0] for node in rotated_matrix))
    min_y = min(list( node[1] for node in rotated_matrix))
    max_y = max(list( node[1] for node in rotated_matrix))
    
    target_section = np.zeros(
        (max_y - min_y + 10, max_x - min_x + 10, 1), np.float32
    )
    
    for node in rotated_matrix:
        if node[2] == 0:
            target_section[node[1] - min_y, node[0] - min_x, 0] = 255
            
    filled_target_section = ndimage.binary_fill_holes(target_section[:,:,0]).astype(int) 
    filled_target_section_props = regionprops(filled_target_section)
    
    for node in filled_target_section_props:
        max_axis = node.major_axis_length
        min_axis = node.minor_axis_length
        
    return max_axis, min_axis, filled_target_section


def culc_mito_center_length(imgs, G, min_point):
    
    """
        imgs : ndarray (Z, Y, X, 1)
        G : ndarray(Z, Y, X)
        max_point : ndarray(Z, Y, X)
    """
    
    
    target_coordinate = (imgs == 255).nonzero()
    target_matrix = [ (x, y, z) for z, y, x in zip(target_coordinate[0], target_coordinate[1], target_coordinate[2]) ]
    
    first_matrix, second_matrix = caluculate_rotate_matrix(min_point, G)
    
    rotated_matrix = [
        np.array(np.dot(
            second_matrix, np.array(np.dot(first_matrix, move_target(target_point, G)), dtype=int)[0]
        ), dtype=int)[0] for target_point in target_matrix
    ]
    
    min_x = min(list( node[0] for node in rotated_matrix))
    max_x = max(list( node[0] for node in rotated_matrix))
    min_y = min(list( node[1] for node in rotated_matrix))
    max_y = max(list( node[1] for node in rotated_matrix))
    min_z = min(list( node[2] for node in rotated_matrix))
    max_z = max(list( node[2] for node in rotated_matrix))
    
    target_sections = np.zeros(
                (max_z - min_z + 10, max_y - min_y + 10, max_x - min_x + 10, 1), np.float32
    )
    print(max_z - min_z)
    for node in rotated_matrix:
        target_sections[node[2] - min_z, node[1] - min_y, node[0] - min_x, 0] = 255

    lst = []
    for z in range(target_sections.shape[0]):
        filled_target_section = ndimage.binary_fill_holes(target_sections[z][:,:,0]).astype(int)
        if np.max(filled_target_section) != 0:
            lst.append(pick_center(filled_target_section) )
            
    
    return lst
    
    

In [ ]:
def culc_distance(a, b):
    
    d = (a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2 + 1
    
    return d ** 0.5

In [ ]:
mito_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/devided_mito/')
connectivity = 6
labels_mito_imgs = cc3d.connected_components((denormalize_y(mito_imgs[:,:,:,0]) == 1).astype(int), connectivity = connectivity)

In [ ]:
result_dct = {}

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):
    target_img = np.where(
        labels_mito_imgs==i,
        1,
        0
    ).astype('uint8')
    if i != 0:
        target_img = target_img[..., np.newaxis]
    
        #V = quantitatives(target_img_)
        V = np.sum(target_img)
        if V > 40:
            S = Quantify_Mito_(target_img)
            result_dct.update({i : {'S' : S, 'V' : V}})

In [ ]:
df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'mito_S', 'mito_V']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'mito_S' : values['S'], 
                    'mito_V' : values['V']
                }
            ),  ignore_index = True
        )  

In [ ]:
#result_dct = {}

for i in tqdm(df.itertuples()):
    mito_No = int(i[1])
    target_img = np.where(
        labels_mito_imgs==mito_No,
        1,
        0
    ).astype('uint8')
    
    g_point = pick_center(target_img)
    
    result_dct[mito_No]['mito_G_point_x'] = g_point[2]
    result_dct[mito_No]['mito_G_point_y'] = g_point[1]
    result_dct[mito_No]['mito_G_point_z'] = g_point[0]

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_G_point_x"] = values["mito_G_point_x"]
    df.loc[df["mito_No"] == key, "mito_G_point_y"] = values["mito_G_point_y"]
    df.loc[df["mito_No"] == key, "mito_G_point_z"] = values["mito_G_point_z"]
    

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    print(target_max_point, target_min_point)
    lst = culc_mito_center_length(target_img, target_g, target_min_point)
    
    length = sum(list( culc_distance(lst[i], lst[i+1])  for i in range(len(lst) - 1) ))
    
    #cropped_001_length_dct.update({
    #    num : {
    #        "mito_max_L" : target_max_L,
    #        "mito_center_L" : length
    #    }
    #})
    result_dct[num]['mito_max_L'] = target_max_L
    result_dct[num]['mito_center_L'] = length

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    max_axis, min_axis, filled_target_section = culc_short_axes(target_img, target_g, target_min_point)
    
    
    result_dct[num]["mito_mid_L"] = max_axis
    result_dct[num]["mito_min_L"] = min_axis
    

In [ ]:
tubular_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_final/tubular_with_postprocessing//')
lamellar_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_001/merged_cristae_final/lamellar_with_postprocessing//')
tubular_imgs = np.where(
    tubular_imgs * 255 * 255  > 127,
    1,
    0
)
lamellar_imgs = np.where(
    lamellar_imgs * 255 * 255  > 127,
    1,
    0
)

cristae_imgs = tubular_imgs * 255  + lamellar_imgs * 255
cristae_imgs = np.where(
    cristae_imgs > 0,
    255,
    0
)
cristae_counters = pick_counters(cristae_imgs[:,:,:,0])

In [ ]:
for i in tqdm(result_dct.keys()):
    target_tubular_img = np.where(
        labels_mito_imgs == i,
        tubular_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    target_lamellar_img = np.where(
        labels_mito_imgs == i,
        lamellar_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    res_obj = culcurate_cristae_SV(target_lamellar_img, target_tubular_img)
    
    """cropped_004_result_dict_.update({
        i : {
            "lamellar_S" : res_obj["lamellar_counters"],
            'lamellar_V' : np.count_nonzero(target_lamellar_img),
            "tubular_S" : res_obj["tubular_counters"],
            'tubular_V' : np.count_nonzero(target_tubular_img),
        }
    })"""
    result_dct[i]["lamellar_S"] = res_obj["lamellar_counters"]
    result_dct[i]["lamellar_V"] = np.count_nonzero(target_lamellar_img)
    result_dct[i]["tubular_S"] = res_obj["tubular_counters"]
    result_dct[i]["tubular_V"] = np.count_nonzero(target_tubular_img)
    

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_max_L"] = values["mito_max_L"]
    df.loc[df["mito_No"] == key, "mito_mid_L"] = values["mito_mid_L"]
    df.loc[df["mito_No"] == key, "mito_min_L"] = values["mito_min_L"]
    df.loc[df["mito_No"] == key, "mito_center_L"] = values["mito_center_L"]
    df.loc[df["mito_No"] == key, "lamellar_S"] = values["lamellar_S"]
    df.loc[df["mito_No"] == key, "lamellar_V"] = values["lamellar_V"]
    df.loc[df["mito_No"] == key, "tubular_S"] = values["tubular_S"]
    df.loc[df["mito_No"] == key, "tubular_V"] = values["tubular_V"]
    

In [ ]:
os.makedirs("H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cropped_001", exist_ok = True)

df.to_csv('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cropped_001/mito_001.csv')

In [ ]:
mito_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/devided_mito/')
connectivity = 6
labels_mito_imgs = cc3d.connected_components((denormalize_y(mito_imgs[:,:,:,0]) == 1).astype(int), connectivity = connectivity)

In [ ]:
result_dct = {}

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):
    target_img = np.where(
        labels_mito_imgs==i,
        1,
        0
    ).astype('uint8')
    if i != 0:
        target_img = target_img[..., np.newaxis]
    
        #V = quantitatives(target_img_)
        V = np.sum(target_img)
        if V > 40:
            S = Quantify_Mito_(target_img)
            result_dct.update({i : {'S' : S, 'V' : V}})

In [ ]:
df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'mito_S', 'mito_V']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'mito_S' : values['S'], 
                    'mito_V' : values['V']
                }
            ),  ignore_index = True
        )  

In [ ]:
#result_dct = {}

for i in tqdm(df.itertuples()):
    mito_No = int(i[1])
    target_img = np.where(
        labels_mito_imgs==mito_No,
        1,
        0
    ).astype('uint8')
    
    g_point = pick_center(target_img)
    
    result_dct[mito_No]['mito_G_point_x'] = g_point[2]
    result_dct[mito_No]['mito_G_point_y'] = g_point[1]
    result_dct[mito_No]['mito_G_point_z'] = g_point[0]

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_G_point_x"] = values["mito_G_point_x"]
    df.loc[df["mito_No"] == key, "mito_G_point_y"] = values["mito_G_point_y"]
    df.loc[df["mito_No"] == key, "mito_G_point_z"] = values["mito_G_point_z"]
    

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    print(target_max_point, target_min_point)
    lst = culc_mito_center_length(target_img, target_g, target_min_point)
    
    length = sum(list( culc_distance(lst[i], lst[i+1])  for i in range(len(lst) - 1) ))
    
    #cropped_001_length_dct.update({
    #    num : {
    #        "mito_max_L" : target_max_L,
    #        "mito_center_L" : length
    #    }
    #})
    result_dct[num]['mito_max_L'] = target_max_L
    result_dct[num]['mito_center_L'] = length

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    max_axis, min_axis, filled_target_section = culc_short_axes(target_img, target_g, target_min_point)
    
    
    result_dct[num]["mito_mid_L"] = max_axis
    result_dct[num]["mito_min_L"] = min_axis
    

In [ ]:
tubular_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_final/tubular_with_postprocessing//')
lamellar_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_002/merged_cristae_final/lamellar_with_postprocessing//')
tubular_imgs = np.where(
    tubular_imgs * 255 * 255  > 127,
    1,
    0
)
lamellar_imgs = np.where(
    lamellar_imgs * 255 * 255  > 127,
    1,
    0
)

cristae_imgs = tubular_imgs * 255  + lamellar_imgs * 255
cristae_imgs = np.where(
    cristae_imgs > 0,
    255,
    0
)
cristae_counters = pick_counters(cristae_imgs[:,:,:,0])

In [ ]:
for i in tqdm(result_dct.keys()):
    target_tubular_img = np.where(
        labels_mito_imgs == i,
        tubular_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    target_lamellar_img = np.where(
        labels_mito_imgs == i,
        lamellar_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    res_obj = culcurate_cristae_SV(target_lamellar_img, target_tubular_img)
    
    """cropped_004_result_dict_.update({
        i : {
            "lamellar_S" : res_obj["lamellar_counters"],
            'lamellar_V' : np.count_nonzero(target_lamellar_img),
            "tubular_S" : res_obj["tubular_counters"],
            'tubular_V' : np.count_nonzero(target_tubular_img),
        }
    })"""
    result_dct[i]["lamellar_S"] = res_obj["lamellar_counters"]
    result_dct[i]["lamellar_V"] = np.count_nonzero(target_lamellar_img)
    result_dct[i]["tubular_S"] = res_obj["tubular_counters"]
    result_dct[i]["tubular_V"] = np.count_nonzero(target_tubular_img)
    

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_max_L"] = values["mito_max_L"]
    df.loc[df["mito_No"] == key, "mito_mid_L"] = values["mito_mid_L"]
    df.loc[df["mito_No"] == key, "mito_min_L"] = values["mito_min_L"]
    df.loc[df["mito_No"] == key, "mito_center_L"] = values["mito_center_L"]
    df.loc[df["mito_No"] == key, "lamellar_S"] = values["lamellar_S"]
    df.loc[df["mito_No"] == key, "lamellar_V"] = values["lamellar_V"]
    df.loc[df["mito_No"] == key, "tubular_S"] = values["tubular_S"]
    df.loc[df["mito_No"] == key, "tubular_V"] = values["tubular_V"]
    

In [ ]:
os.makedirs("H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cropped_002", exist_ok = True)

df.to_csv('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cropped_002/mito_001.csv')

In [ ]:
mito_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/devided_mito/')
connectivity = 6
labels_mito_imgs = cc3d.connected_components((denormalize_y(mito_imgs[:,:,:,0]) == 1).astype(int), connectivity = connectivity)

In [ ]:
result_dct = {}

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):
    target_img = np.where(
        labels_mito_imgs==i,
        1,
        0
    ).astype('uint8')
    if i != 0:
        target_img = target_img[..., np.newaxis]
    
        #V = quantitatives(target_img_)
        V = np.sum(target_img)
        if V > 40:
            S = Quantify_Mito_(target_img)
            result_dct.update({i : {'S' : S, 'V' : V}})

In [ ]:
df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'mito_S', 'mito_V']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'mito_S' : values['S'], 
                    'mito_V' : values['V']
                }
            ),  ignore_index = True
        )  

In [ ]:
#result_dct = {}

for i in tqdm(df.itertuples()):
    mito_No = int(i[1])
    target_img = np.where(
        labels_mito_imgs==mito_No,
        1,
        0
    ).astype('uint8')
    
    g_point = pick_center(target_img)
    
    result_dct[mito_No]['mito_G_point_x'] = g_point[2]
    result_dct[mito_No]['mito_G_point_y'] = g_point[1]
    result_dct[mito_No]['mito_G_point_z'] = g_point[0]

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_G_point_x"] = values["mito_G_point_x"]
    df.loc[df["mito_No"] == key, "mito_G_point_y"] = values["mito_G_point_y"]
    df.loc[df["mito_No"] == key, "mito_G_point_z"] = values["mito_G_point_z"]
    

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    print(target_max_point, target_min_point)
    lst = culc_mito_center_length(target_img, target_g, target_min_point)
    
    length = sum(list( culc_distance(lst[i], lst[i+1])  for i in range(len(lst) - 1) ))
    
    #cropped_001_length_dct.update({
    #    num : {
    #        "mito_max_L" : target_max_L,
    #        "mito_center_L" : length
    #    }
    #})
    result_dct[num]['mito_max_L'] = target_max_L
    result_dct[num]['mito_center_L'] = length

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    max_axis, min_axis, filled_target_section = culc_short_axes(target_img, target_g, target_min_point)
    
    
    result_dct[num]["mito_mid_L"] = max_axis
    result_dct[num]["mito_min_L"] = min_axis
    

In [ ]:
tubular_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_final/tubular_with_postprocessing//')
lamellar_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_003/merged_cristae_final/lamellar_with_postprocessing//')
tubular_imgs = np.where(
    tubular_imgs * 255 * 255  > 127,
    1,
    0
)
lamellar_imgs = np.where(
    lamellar_imgs * 255 * 255  > 127,
    1,
    0
)

cristae_imgs = tubular_imgs * 255  + lamellar_imgs * 255
cristae_imgs = np.where(
    cristae_imgs > 0,
    255,
    0
)
cristae_counters = pick_counters(cristae_imgs[:,:,:,0])

In [ ]:
for i in tqdm(result_dct.keys()):
    target_tubular_img = np.where(
        labels_mito_imgs == i,
        tubular_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    target_lamellar_img = np.where(
        labels_mito_imgs == i,
        lamellar_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    res_obj = culcurate_cristae_SV(target_lamellar_img, target_tubular_img)
    
    """cropped_004_result_dict_.update({
        i : {
            "lamellar_S" : res_obj["lamellar_counters"],
            'lamellar_V' : np.count_nonzero(target_lamellar_img),
            "tubular_S" : res_obj["tubular_counters"],
            'tubular_V' : np.count_nonzero(target_tubular_img),
        }
    })"""
    result_dct[i]["lamellar_S"] = res_obj["lamellar_counters"]
    result_dct[i]["lamellar_V"] = np.count_nonzero(target_lamellar_img)
    result_dct[i]["tubular_S"] = res_obj["tubular_counters"]
    result_dct[i]["tubular_V"] = np.count_nonzero(target_tubular_img)
    

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_max_L"] = values["mito_max_L"]
    df.loc[df["mito_No"] == key, "mito_mid_L"] = values["mito_mid_L"]
    df.loc[df["mito_No"] == key, "mito_min_L"] = values["mito_min_L"]
    df.loc[df["mito_No"] == key, "mito_center_L"] = values["mito_center_L"]
    df.loc[df["mito_No"] == key, "lamellar_S"] = values["lamellar_S"]
    df.loc[df["mito_No"] == key, "lamellar_V"] = values["lamellar_V"]
    df.loc[df["mito_No"] == key, "tubular_S"] = values["tubular_S"]
    df.loc[df["mito_No"] == key, "tubular_V"] = values["tubular_V"]
    

In [ ]:
os.makedirs("H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cropped_003", exist_ok = True)

df.to_csv('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cropped_003/mito_001.csv')

In [ ]:
mito_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/devided_mito/')
connectivity = 6
labels_mito_imgs = cc3d.connected_components((denormalize_y(mito_imgs[:,:,:,0]) == 1).astype(int), connectivity = connectivity)

In [ ]:
result_dct = {}

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):
    target_img = np.where(
        labels_mito_imgs==i,
        1,
        0
    ).astype('uint8')
    if i != 0:
        target_img = target_img[..., np.newaxis]
    
        #V = quantitatives(target_img_)
        V = np.sum(target_img)
        if V > 40:
            S = Quantify_Mito_(target_img)
            result_dct.update({i : {'S' : S, 'V' : V}})

In [ ]:
df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'mito_S', 'mito_V']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'mito_S' : values['S'], 
                    'mito_V' : values['V']
                }
            ),  ignore_index = True
        )  

In [ ]:
#result_dct = {}

for i in tqdm(df.itertuples()):
    mito_No = int(i[1])
    target_img = np.where(
        labels_mito_imgs==mito_No,
        1,
        0
    ).astype('uint8')
    
    g_point = pick_center(target_img)
    
    result_dct[mito_No]['mito_G_point_x'] = g_point[2]
    result_dct[mito_No]['mito_G_point_y'] = g_point[1]
    result_dct[mito_No]['mito_G_point_z'] = g_point[0]

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_G_point_x"] = values["mito_G_point_x"]
    df.loc[df["mito_No"] == key, "mito_G_point_y"] = values["mito_G_point_y"]
    df.loc[df["mito_No"] == key, "mito_G_point_z"] = values["mito_G_point_z"]
    

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    print(target_max_point, target_min_point)
    lst = culc_mito_center_length(target_img, target_g, target_min_point)
    
    length = sum(list( culc_distance(lst[i], lst[i+1])  for i in range(len(lst) - 1) ))
    
    #cropped_001_length_dct.update({
    #    num : {
    #        "mito_max_L" : target_max_L,
    #        "mito_center_L" : length
    #    }
    #})
    result_dct[num]['mito_max_L'] = target_max_L
    result_dct[num]['mito_center_L'] = length

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    max_axis, min_axis, filled_target_section = culc_short_axes(target_img, target_g, target_min_point)
    
    
    result_dct[num]["mito_mid_L"] = max_axis
    result_dct[num]["mito_min_L"] = min_axis
    

In [ ]:
tubular_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_final/tubular_with_postprocessing//')
lamellar_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/merged_cristae_final/lamellar_with_postprocessing//')
tubular_imgs = np.where(
    tubular_imgs * 255 * 255  > 127,
    1,
    0
)
lamellar_imgs = np.where(
    lamellar_imgs * 255 * 255  > 127,
    1,
    0
)

cristae_imgs = tubular_imgs * 255  + lamellar_imgs * 255
cristae_imgs = np.where(
    cristae_imgs > 0,
    255,
    0
)
cristae_counters = pick_counters(cristae_imgs[:,:,:,0])

In [ ]:
for i in tqdm(result_dct.keys()):
    target_tubular_img = np.where(
        labels_mito_imgs == i,
        tubular_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    target_lamellar_img = np.where(
        labels_mito_imgs == i,
        lamellar_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    res_obj = culcurate_cristae_SV(target_lamellar_img, target_tubular_img)
    
    """cropped_004_result_dict_.update({
        i : {
            "lamellar_S" : res_obj["lamellar_counters"],
            'lamellar_V' : np.count_nonzero(target_lamellar_img),
            "tubular_S" : res_obj["tubular_counters"],
            'tubular_V' : np.count_nonzero(target_tubular_img),
        }
    })"""
    result_dct[i]["lamellar_S"] = res_obj["lamellar_counters"]
    result_dct[i]["lamellar_V"] = np.count_nonzero(target_lamellar_img)
    result_dct[i]["tubular_S"] = res_obj["tubular_counters"]
    result_dct[i]["tubular_V"] = np.count_nonzero(target_tubular_img)
    

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_max_L"] = values["mito_max_L"]
    df.loc[df["mito_No"] == key, "mito_mid_L"] = values["mito_mid_L"]
    df.loc[df["mito_No"] == key, "mito_min_L"] = values["mito_min_L"]
    df.loc[df["mito_No"] == key, "mito_center_L"] = values["mito_center_L"]
    df.loc[df["mito_No"] == key, "lamellar_S"] = values["lamellar_S"]
    df.loc[df["mito_No"] == key, "lamellar_V"] = values["lamellar_V"]
    df.loc[df["mito_No"] == key, "tubular_S"] = values["tubular_S"]
    df.loc[df["mito_No"] == key, "tubular_V"] = values["tubular_V"]
    

In [ ]:
os.makedirs("H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cropped_004", exist_ok = True)

df.to_csv('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cropped_004/mito_001.csv')

In [ ]:
mito_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/devided_mito/')
connectivity = 6
labels_mito_imgs = cc3d.connected_components((denormalize_y(mito_imgs[:,:,:,0]) == 1).astype(int), connectivity = connectivity)

In [ ]:
result_dct = {}

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):
    target_img = np.where(
        labels_mito_imgs==i,
        1,
        0
    ).astype('uint8')
    if i != 0:
        target_img = target_img[..., np.newaxis]
    
        #V = quantitatives(target_img_)
        V = np.sum(target_img)
        if V > 40:
            S = Quantify_Mito_(target_img)
            result_dct.update({i : {'S' : S, 'V' : V}})

In [ ]:
df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'mito_S', 'mito_V']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'mito_S' : values['S'], 
                    'mito_V' : values['V']
                }
            ),  ignore_index = True
        )  

In [ ]:
#result_dct = {}

for i in tqdm(df.itertuples()):
    mito_No = int(i[1])
    target_img = np.where(
        labels_mito_imgs==mito_No,
        1,
        0
    ).astype('uint8')
    
    g_point = pick_center(target_img)
    
    result_dct[mito_No]['mito_G_point_x'] = g_point[2]
    result_dct[mito_No]['mito_G_point_y'] = g_point[1]
    result_dct[mito_No]['mito_G_point_z'] = g_point[0]

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_G_point_x"] = values["mito_G_point_x"]
    df.loc[df["mito_No"] == key, "mito_G_point_y"] = values["mito_G_point_y"]
    df.loc[df["mito_No"] == key, "mito_G_point_z"] = values["mito_G_point_z"]
    

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    print(target_max_point, target_min_point)
    lst = culc_mito_center_length(target_img, target_g, target_min_point)
    
    length = sum(list( culc_distance(lst[i], lst[i+1])  for i in range(len(lst) - 1) ))
    
    #cropped_001_length_dct.update({
    #    num : {
    #        "mito_max_L" : target_max_L,
    #        "mito_center_L" : length
    #    }
    #})
    result_dct[num]['mito_max_L'] = target_max_L
    result_dct[num]['mito_center_L'] = length

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    max_axis, min_axis, filled_target_section = culc_short_axes(target_img, target_g, target_min_point)
    
    
    result_dct[num]["mito_mid_L"] = max_axis
    result_dct[num]["mito_min_L"] = min_axis
    

In [ ]:
tubular_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_final/tubular_with_postprocessing//')
lamellar_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_cristae_final/lamellar_with_postprocessing//')
tubular_imgs = np.where(
    tubular_imgs * 255 * 255  > 127,
    1,
    0
)
lamellar_imgs = np.where(
    lamellar_imgs * 255 * 255  > 127,
    1,
    0
)

cristae_imgs = tubular_imgs * 255  + lamellar_imgs * 255
cristae_imgs = np.where(
    cristae_imgs > 0,
    255,
    0
)
cristae_counters = pick_counters(cristae_imgs[:,:,:,0])

In [ ]:
for i in tqdm(result_dct.keys()):
    target_tubular_img = np.where(
        labels_mito_imgs == i,
        tubular_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    target_lamellar_img = np.where(
        labels_mito_imgs == i,
        lamellar_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    res_obj = culcurate_cristae_SV(target_lamellar_img, target_tubular_img)
    
    """cropped_004_result_dict_.update({
        i : {
            "lamellar_S" : res_obj["lamellar_counters"],
            'lamellar_V' : np.count_nonzero(target_lamellar_img),
            "tubular_S" : res_obj["tubular_counters"],
            'tubular_V' : np.count_nonzero(target_tubular_img),
        }
    })"""
    result_dct[i]["lamellar_S"] = res_obj["lamellar_counters"]
    result_dct[i]["lamellar_V"] = np.count_nonzero(target_lamellar_img)
    result_dct[i]["tubular_S"] = res_obj["tubular_counters"]
    result_dct[i]["tubular_V"] = np.count_nonzero(target_tubular_img)
    

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_max_L"] = values["mito_max_L"]
    df.loc[df["mito_No"] == key, "mito_mid_L"] = values["mito_mid_L"]
    df.loc[df["mito_No"] == key, "mito_min_L"] = values["mito_min_L"]
    df.loc[df["mito_No"] == key, "mito_center_L"] = values["mito_center_L"]
    df.loc[df["mito_No"] == key, "lamellar_S"] = values["lamellar_S"]
    df.loc[df["mito_No"] == key, "lamellar_V"] = values["lamellar_V"]
    df.loc[df["mito_No"] == key, "tubular_S"] = values["tubular_S"]
    df.loc[df["mito_No"] == key, "tubular_V"] = values["tubular_V"]
    

In [ ]:
os.makedirs("H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cropped_005", exist_ok = True)

df.to_csv('H:/DeepLearningData/research_010_NIH3T3/shCtrl_003/data/cropped_005/mito_001.csv')

In [ ]:
mito_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/devided_mito/')
connectivity = 6
labels_mito_imgs = cc3d.connected_components((denormalize_y(mito_imgs[:,:,:,0]) == 1).astype(int), connectivity = connectivity)

In [ ]:
result_dct = {}

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):
    target_img = np.where(
        labels_mito_imgs==i,
        1,
        0
    ).astype('uint8')
    if i != 0:
        target_img = target_img[..., np.newaxis]
    
        #V = quantitatives(target_img_)
        V = np.sum(target_img)
        if V > 40:
            S = Quantify_Mito_(target_img)
            result_dct.update({i : {'S' : S, 'V' : V}})

In [ ]:
df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'mito_S', 'mito_V']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'mito_S' : values['S'], 
                    'mito_V' : values['V']
                }
            ),  ignore_index = True
        )  

In [ ]:
#result_dct = {}

for i in tqdm(df.itertuples()):
    mito_No = int(i[1])
    target_img = np.where(
        labels_mito_imgs==mito_No,
        1,
        0
    ).astype('uint8')
    
    g_point = pick_center(target_img)
    
    result_dct[mito_No]['mito_G_point_x'] = g_point[2]
    result_dct[mito_No]['mito_G_point_y'] = g_point[1]
    result_dct[mito_No]['mito_G_point_z'] = g_point[0]

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_G_point_x"] = values["mito_G_point_x"]
    df.loc[df["mito_No"] == key, "mito_G_point_y"] = values["mito_G_point_y"]
    df.loc[df["mito_No"] == key, "mito_G_point_z"] = values["mito_G_point_z"]
    

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    print(target_max_point, target_min_point)
    lst = culc_mito_center_length(target_img, target_g, target_min_point)
    
    length = sum(list( culc_distance(lst[i], lst[i+1])  for i in range(len(lst) - 1) ))
    
    #cropped_001_length_dct.update({
    #    num : {
    #        "mito_max_L" : target_max_L,
    #        "mito_center_L" : length
    #    }
    #})
    result_dct[num]['mito_max_L'] = target_max_L
    result_dct[num]['mito_center_L'] = length

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    max_axis, min_axis, filled_target_section = culc_short_axes(target_img, target_g, target_min_point)
    
    
    result_dct[num]["mito_mid_L"] = max_axis
    result_dct[num]["mito_min_L"] = min_axis
    

In [ ]:
tubular_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_final/tubular_with_postprocessing//')
lamellar_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_001/merged_cristae_final/lamellar_with_postprocessing//')
tubular_imgs = np.where(
    tubular_imgs * 255 * 255  > 127,
    1,
    0
)
lamellar_imgs = np.where(
    lamellar_imgs * 255 * 255  > 127,
    1,
    0
)

cristae_imgs = tubular_imgs * 255  + lamellar_imgs * 255
cristae_imgs = np.where(
    cristae_imgs > 0,
    255,
    0
)
cristae_counters = pick_counters(cristae_imgs[:,:,:,0])

In [ ]:
for i in tqdm(result_dct.keys()):
    target_tubular_img = np.where(
        labels_mito_imgs == i,
        tubular_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    target_lamellar_img = np.where(
        labels_mito_imgs == i,
        lamellar_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    res_obj = culcurate_cristae_SV(target_lamellar_img, target_tubular_img)
    
    """cropped_004_result_dict_.update({
        i : {
            "lamellar_S" : res_obj["lamellar_counters"],
            'lamellar_V' : np.count_nonzero(target_lamellar_img),
            "tubular_S" : res_obj["tubular_counters"],
            'tubular_V' : np.count_nonzero(target_tubular_img),
        }
    })"""
    result_dct[i]["lamellar_S"] = res_obj["lamellar_counters"]
    result_dct[i]["lamellar_V"] = np.count_nonzero(target_lamellar_img)
    result_dct[i]["tubular_S"] = res_obj["tubular_counters"]
    result_dct[i]["tubular_V"] = np.count_nonzero(target_tubular_img)
    

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_max_L"] = values["mito_max_L"]
    df.loc[df["mito_No"] == key, "mito_mid_L"] = values["mito_mid_L"]
    df.loc[df["mito_No"] == key, "mito_min_L"] = values["mito_min_L"]
    df.loc[df["mito_No"] == key, "mito_center_L"] = values["mito_center_L"]
    df.loc[df["mito_No"] == key, "lamellar_S"] = values["lamellar_S"]
    df.loc[df["mito_No"] == key, "lamellar_V"] = values["lamellar_V"]
    df.loc[df["mito_No"] == key, "tubular_S"] = values["tubular_S"]
    df.loc[df["mito_No"] == key, "tubular_V"] = values["tubular_V"]
    

In [ ]:
os.makedirs("H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_001", exist_ok = True)

df.to_csv('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_001/mito_001.csv')

In [ ]:
mito_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/devided_mito/')
connectivity = 6
labels_mito_imgs = cc3d.connected_components((denormalize_y(mito_imgs[:,:,:,0]) == 1).astype(int), connectivity = connectivity)

In [ ]:
result_dct = {}

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):
    target_img = np.where(
        labels_mito_imgs==i,
        1,
        0
    ).astype('uint8')
    if i != 0:
        target_img = target_img[..., np.newaxis]
    
        #V = quantitatives(target_img_)
        V = np.sum(target_img)
        if V > 40:
            S = Quantify_Mito_(target_img)
            result_dct.update({i : {'S' : S, 'V' : V}})

In [ ]:
df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'mito_S', 'mito_V']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'mito_S' : values['S'], 
                    'mito_V' : values['V']
                }
            ),  ignore_index = True
        )  

In [ ]:
#result_dct = {}

for i in tqdm(df.itertuples()):
    mito_No = int(i[1])
    target_img = np.where(
        labels_mito_imgs==mito_No,
        1,
        0
    ).astype('uint8')
    
    g_point = pick_center(target_img)
    
    result_dct[mito_No]['mito_G_point_x'] = g_point[2]
    result_dct[mito_No]['mito_G_point_y'] = g_point[1]
    result_dct[mito_No]['mito_G_point_z'] = g_point[0]

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_G_point_x"] = values["mito_G_point_x"]
    df.loc[df["mito_No"] == key, "mito_G_point_y"] = values["mito_G_point_y"]
    df.loc[df["mito_No"] == key, "mito_G_point_z"] = values["mito_G_point_z"]
    

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    print(target_max_point, target_min_point)
    lst = culc_mito_center_length(target_img, target_g, target_min_point)
    
    length = sum(list( culc_distance(lst[i], lst[i+1])  for i in range(len(lst) - 1) ))
    
    #cropped_001_length_dct.update({
    #    num : {
    #        "mito_max_L" : target_max_L,
    #        "mito_center_L" : length
    #    }
    #})
    result_dct[num]['mito_max_L'] = target_max_L
    result_dct[num]['mito_center_L'] = length

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    max_axis, min_axis, filled_target_section = culc_short_axes(target_img, target_g, target_min_point)
    
    
    result_dct[num]["mito_mid_L"] = max_axis
    result_dct[num]["mito_min_L"] = min_axis
    

In [ ]:
tubular_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_final/tubular_with_postprocessing//')
lamellar_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_002/merged_cristae_final/lamellar_with_postprocessing//')
tubular_imgs = np.where(
    tubular_imgs * 255 * 255  > 127,
    1,
    0
)
lamellar_imgs = np.where(
    lamellar_imgs * 255 * 255  > 127,
    1,
    0
)

cristae_imgs = tubular_imgs * 255  + lamellar_imgs * 255
cristae_imgs = np.where(
    cristae_imgs > 0,
    255,
    0
)
cristae_counters = pick_counters(cristae_imgs[:,:,:,0])

In [ ]:
for i in tqdm(result_dct.keys()):
    target_tubular_img = np.where(
        labels_mito_imgs == i,
        tubular_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    target_lamellar_img = np.where(
        labels_mito_imgs == i,
        lamellar_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    res_obj = culcurate_cristae_SV(target_lamellar_img, target_tubular_img)
    
    """cropped_004_result_dict_.update({
        i : {
            "lamellar_S" : res_obj["lamellar_counters"],
            'lamellar_V' : np.count_nonzero(target_lamellar_img),
            "tubular_S" : res_obj["tubular_counters"],
            'tubular_V' : np.count_nonzero(target_tubular_img),
        }
    })"""
    result_dct[i]["lamellar_S"] = res_obj["lamellar_counters"]
    result_dct[i]["lamellar_V"] = np.count_nonzero(target_lamellar_img)
    result_dct[i]["tubular_S"] = res_obj["tubular_counters"]
    result_dct[i]["tubular_V"] = np.count_nonzero(target_tubular_img)
    

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_max_L"] = values["mito_max_L"]
    df.loc[df["mito_No"] == key, "mito_mid_L"] = values["mito_mid_L"]
    df.loc[df["mito_No"] == key, "mito_min_L"] = values["mito_min_L"]
    df.loc[df["mito_No"] == key, "mito_center_L"] = values["mito_center_L"]
    df.loc[df["mito_No"] == key, "lamellar_S"] = values["lamellar_S"]
    df.loc[df["mito_No"] == key, "lamellar_V"] = values["lamellar_V"]
    df.loc[df["mito_No"] == key, "tubular_S"] = values["tubular_S"]
    df.loc[df["mito_No"] == key, "tubular_V"] = values["tubular_V"]
    

In [ ]:
os.makedirs("H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_002", exist_ok = True)

df.to_csv('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_002/mito_001.csv')

In [ ]:
mito_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/devided_mito/')
connectivity = 6
labels_mito_imgs = cc3d.connected_components((denormalize_y(mito_imgs[:,:,:,0]) == 1).astype(int), connectivity = connectivity)

In [ ]:
result_dct = {}

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):
    target_img = np.where(
        labels_mito_imgs==i,
        1,
        0
    ).astype('uint8')
    if i != 0:
        target_img = target_img[..., np.newaxis]
    
        #V = quantitatives(target_img_)
        V = np.sum(target_img)
        if V > 40:
            S = Quantify_Mito_(target_img)
            result_dct.update({i : {'S' : S, 'V' : V}})

In [ ]:
df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'mito_S', 'mito_V']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'mito_S' : values['S'], 
                    'mito_V' : values['V']
                }
            ),  ignore_index = True
        )  

In [ ]:
#result_dct = {}

for i in tqdm(df.itertuples()):
    mito_No = int(i[1])
    target_img = np.where(
        labels_mito_imgs==mito_No,
        1,
        0
    ).astype('uint8')
    
    g_point = pick_center(target_img)
    
    result_dct[mito_No]['mito_G_point_x'] = g_point[2]
    result_dct[mito_No]['mito_G_point_y'] = g_point[1]
    result_dct[mito_No]['mito_G_point_z'] = g_point[0]

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_G_point_x"] = values["mito_G_point_x"]
    df.loc[df["mito_No"] == key, "mito_G_point_y"] = values["mito_G_point_y"]
    df.loc[df["mito_No"] == key, "mito_G_point_z"] = values["mito_G_point_z"]
    

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    print(target_max_point, target_min_point)
    lst = culc_mito_center_length(target_img, target_g, target_min_point)
    
    length = sum(list( culc_distance(lst[i], lst[i+1])  for i in range(len(lst) - 1) ))
    
    #cropped_001_length_dct.update({
    #    num : {
    #        "mito_max_L" : target_max_L,
    #        "mito_center_L" : length
    #    }
    #})
    result_dct[num]['mito_max_L'] = target_max_L
    result_dct[num]['mito_center_L'] = length

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    max_axis, min_axis, filled_target_section = culc_short_axes(target_img, target_g, target_min_point)
    
    
    result_dct[num]["mito_mid_L"] = max_axis
    result_dct[num]["mito_min_L"] = min_axis
    

In [ ]:
tubular_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_final/tubular_with_postprocessing//')
lamellar_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_cristae_final/lamellar_with_postprocessing//')
tubular_imgs = np.where(
    tubular_imgs * 255 * 255  > 127,
    1,
    0
)
lamellar_imgs = np.where(
    lamellar_imgs * 255 * 255  > 127,
    1,
    0
)

cristae_imgs = tubular_imgs * 255  + lamellar_imgs * 255
cristae_imgs = np.where(
    cristae_imgs > 0,
    255,
    0
)
cristae_counters = pick_counters(cristae_imgs[:,:,:,0])

In [ ]:
for i in tqdm(result_dct.keys()):
    target_tubular_img = np.where(
        labels_mito_imgs == i,
        tubular_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    target_lamellar_img = np.where(
        labels_mito_imgs == i,
        lamellar_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    res_obj = culcurate_cristae_SV(target_lamellar_img, target_tubular_img)
    
    """cropped_004_result_dict_.update({
        i : {
            "lamellar_S" : res_obj["lamellar_counters"],
            'lamellar_V' : np.count_nonzero(target_lamellar_img),
            "tubular_S" : res_obj["tubular_counters"],
            'tubular_V' : np.count_nonzero(target_tubular_img),
        }
    })"""
    result_dct[i]["lamellar_S"] = res_obj["lamellar_counters"]
    result_dct[i]["lamellar_V"] = np.count_nonzero(target_lamellar_img)
    result_dct[i]["tubular_S"] = res_obj["tubular_counters"]
    result_dct[i]["tubular_V"] = np.count_nonzero(target_tubular_img)
    

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_max_L"] = values["mito_max_L"]
    df.loc[df["mito_No"] == key, "mito_mid_L"] = values["mito_mid_L"]
    df.loc[df["mito_No"] == key, "mito_min_L"] = values["mito_min_L"]
    df.loc[df["mito_No"] == key, "mito_center_L"] = values["mito_center_L"]
    df.loc[df["mito_No"] == key, "lamellar_S"] = values["lamellar_S"]
    df.loc[df["mito_No"] == key, "lamellar_V"] = values["lamellar_V"]
    df.loc[df["mito_No"] == key, "tubular_S"] = values["tubular_S"]
    df.loc[df["mito_No"] == key, "tubular_V"] = values["tubular_V"]
    

In [ ]:
os.makedirs("H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_003", exist_ok = True)

df.to_csv('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_003/mito_001.csv')

In [ ]:
mito_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/devided_mito/')
connectivity = 6
labels_mito_imgs = cc3d.connected_components((denormalize_y(mito_imgs[:,:,:,0]) == 1).astype(int), connectivity = connectivity)

In [ ]:
result_dct = {}

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):
    target_img = np.where(
        labels_mito_imgs==i,
        1,
        0
    ).astype('uint8')
    if i != 0:
        target_img = target_img[..., np.newaxis]
    
        #V = quantitatives(target_img_)
        V = np.sum(target_img)
        if V > 40:
            S = Quantify_Mito_(target_img)
            result_dct.update({i : {'S' : S, 'V' : V}})

In [ ]:
df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'mito_S', 'mito_V']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'mito_S' : values['S'], 
                    'mito_V' : values['V']
                }
            ),  ignore_index = True
        )  

In [ ]:
#result_dct = {}

for i in tqdm(df.itertuples()):
    mito_No = int(i[1])
    target_img = np.where(
        labels_mito_imgs==mito_No,
        1,
        0
    ).astype('uint8')
    
    g_point = pick_center(target_img)
    
    result_dct[mito_No]['mito_G_point_x'] = g_point[2]
    result_dct[mito_No]['mito_G_point_y'] = g_point[1]
    result_dct[mito_No]['mito_G_point_z'] = g_point[0]

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_G_point_x"] = values["mito_G_point_x"]
    df.loc[df["mito_No"] == key, "mito_G_point_y"] = values["mito_G_point_y"]
    df.loc[df["mito_No"] == key, "mito_G_point_z"] = values["mito_G_point_z"]
    

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    print(target_max_point, target_min_point)
    lst = culc_mito_center_length(target_img, target_g, target_min_point)
    
    length = sum(list( culc_distance(lst[i], lst[i+1])  for i in range(len(lst) - 1) ))
    
    #cropped_001_length_dct.update({
    #    num : {
    #        "mito_max_L" : target_max_L,
    #        "mito_center_L" : length
    #    }
    #})
    result_dct[num]['mito_max_L'] = target_max_L
    result_dct[num]['mito_center_L'] = length

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    max_axis, min_axis, filled_target_section = culc_short_axes(target_img, target_g, target_min_point)
    
    
    result_dct[num]["mito_mid_L"] = max_axis
    result_dct[num]["mito_min_L"] = min_axis
    

In [ ]:
tubular_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_final/tubular_with_postprocessing//')
lamellar_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_cristae_final/lamellar_with_postprocessing//')
tubular_imgs = np.where(
    tubular_imgs * 255 * 255  > 127,
    1,
    0
)
lamellar_imgs = np.where(
    lamellar_imgs * 255 * 255  > 127,
    1,
    0
)

cristae_imgs = tubular_imgs * 255  + lamellar_imgs * 255
cristae_imgs = np.where(
    cristae_imgs > 0,
    255,
    0
)
cristae_counters = pick_counters(cristae_imgs[:,:,:,0])

In [ ]:
for i in tqdm(result_dct.keys()):
    target_tubular_img = np.where(
        labels_mito_imgs == i,
        tubular_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    target_lamellar_img = np.where(
        labels_mito_imgs == i,
        lamellar_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    res_obj = culcurate_cristae_SV(target_lamellar_img, target_tubular_img)
    
    """cropped_004_result_dict_.update({
        i : {
            "lamellar_S" : res_obj["lamellar_counters"],
            'lamellar_V' : np.count_nonzero(target_lamellar_img),
            "tubular_S" : res_obj["tubular_counters"],
            'tubular_V' : np.count_nonzero(target_tubular_img),
        }
    })"""
    result_dct[i]["lamellar_S"] = res_obj["lamellar_counters"]
    result_dct[i]["lamellar_V"] = np.count_nonzero(target_lamellar_img)
    result_dct[i]["tubular_S"] = res_obj["tubular_counters"]
    result_dct[i]["tubular_V"] = np.count_nonzero(target_tubular_img)
    

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_max_L"] = values["mito_max_L"]
    df.loc[df["mito_No"] == key, "mito_mid_L"] = values["mito_mid_L"]
    df.loc[df["mito_No"] == key, "mito_min_L"] = values["mito_min_L"]
    df.loc[df["mito_No"] == key, "mito_center_L"] = values["mito_center_L"]
    df.loc[df["mito_No"] == key, "lamellar_S"] = values["lamellar_S"]
    df.loc[df["mito_No"] == key, "lamellar_V"] = values["lamellar_V"]
    df.loc[df["mito_No"] == key, "tubular_S"] = values["tubular_S"]
    df.loc[df["mito_No"] == key, "tubular_V"] = values["tubular_V"]
    

In [ ]:
os.makedirs("H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_004", exist_ok = True)

df.to_csv('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_004/mito_001.csv')

In [ ]:
mito_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/devided_mito/')
connectivity = 6
labels_mito_imgs = cc3d.connected_components((denormalize_y(mito_imgs[:,:,:,0]) == 1).astype(int), connectivity = connectivity)

In [ ]:
result_dct = {}

for i in tqdm(range(int(np.max(labels_mito_imgs) + 1))):
    target_img = np.where(
        labels_mito_imgs==i,
        1,
        0
    ).astype('uint8')
    if i != 0:
        target_img = target_img[..., np.newaxis]
    
        #V = quantitatives(target_img_)
        V = np.sum(target_img)
        if V > 40:
            S = Quantify_Mito_(target_img)
            result_dct.update({i : {'S' : S, 'V' : V}})

In [ ]:
df = pd.DataFrame(
    index=[], 
    columns=['mito_No', 'mito_S', 'mito_V']
)

for key, values in result_dct.items():
        df = df.append(
            pd.Series(
                {
                    'mito_No' : key,
                    'mito_S' : values['S'], 
                    'mito_V' : values['V']
                }
            ),  ignore_index = True
        )  

In [ ]:
#result_dct = {}

for i in tqdm(df.itertuples()):
    mito_No = int(i[1])
    target_img = np.where(
        labels_mito_imgs==mito_No,
        1,
        0
    ).astype('uint8')
    
    g_point = pick_center(target_img)
    
    result_dct[mito_No]['mito_G_point_x'] = g_point[2]
    result_dct[mito_No]['mito_G_point_y'] = g_point[1]
    result_dct[mito_No]['mito_G_point_z'] = g_point[0]

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_G_point_x"] = values["mito_G_point_x"]
    df.loc[df["mito_No"] == key, "mito_G_point_y"] = values["mito_G_point_y"]
    df.loc[df["mito_No"] == key, "mito_G_point_z"] = values["mito_G_point_z"]
    

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    print(target_max_point, target_min_point)
    lst = culc_mito_center_length(target_img, target_g, target_min_point)
    
    length = sum(list( culc_distance(lst[i], lst[i+1])  for i in range(len(lst) - 1) ))
    
    #cropped_001_length_dct.update({
    #    num : {
    #        "mito_max_L" : target_max_L,
    #        "mito_center_L" : length
    #    }
    #})
    result_dct[num]['mito_max_L'] = target_max_L
    result_dct[num]['mito_center_L'] = length

In [ ]:
for num in tqdm(range(np.max(labels_mito_imgs) + 1)):
    if num == 0:
        continue
    target_img = np.where(
        labels_mito_imgs==num,
        255,
        0
    ).astype('uint8')
    
    target_g  = culc_gravity(target_img)
    target_max_L, target_max_point, target_min_point = culc_axis(target_img, target_g)
    target_max_L = sum(list( (target_max_point[i] - target_min_point[i]) ** 2 for i in range(3) )) ** 0.5
    max_axis, min_axis, filled_target_section = culc_short_axes(target_img, target_g, target_min_point)
    
    
    result_dct[num]["mito_mid_L"] = max_axis
    result_dct[num]["mito_min_L"] = min_axis
    

In [ ]:
tubular_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_final/tubular_with_postprocessing//')
lamellar_imgs, _ = load_Y_gray('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_cristae_final/lamellar_with_postprocessing//')
tubular_imgs = np.where(
    tubular_imgs * 255 * 255  > 127,
    1,
    0
)
lamellar_imgs = np.where(
    lamellar_imgs * 255 * 255  > 127,
    1,
    0
)

cristae_imgs = tubular_imgs * 255  + lamellar_imgs * 255
cristae_imgs = np.where(
    cristae_imgs > 0,
    255,
    0
)
cristae_counters = pick_counters(cristae_imgs[:,:,:,0])

In [ ]:
for i in tqdm(result_dct.keys()):
    target_tubular_img = np.where(
        labels_mito_imgs == i,
        tubular_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    target_lamellar_img = np.where(
        labels_mito_imgs == i,
        lamellar_imgs[:,:,:,0] * 255,
        0
    ).astype("uint8")
    
    res_obj = culcurate_cristae_SV(target_lamellar_img, target_tubular_img)
    
    """cropped_004_result_dict_.update({
        i : {
            "lamellar_S" : res_obj["lamellar_counters"],
            'lamellar_V' : np.count_nonzero(target_lamellar_img),
            "tubular_S" : res_obj["tubular_counters"],
            'tubular_V' : np.count_nonzero(target_tubular_img),
        }
    })"""
    result_dct[i]["lamellar_S"] = res_obj["lamellar_counters"]
    result_dct[i]["lamellar_V"] = np.count_nonzero(target_lamellar_img)
    result_dct[i]["tubular_S"] = res_obj["tubular_counters"]
    result_dct[i]["tubular_V"] = np.count_nonzero(target_tubular_img)
    

In [ ]:
for key, values in result_dct.items():
    #print(key)
    df.loc[df["mito_No"] == key, "mito_max_L"] = values["mito_max_L"]
    df.loc[df["mito_No"] == key, "mito_mid_L"] = values["mito_mid_L"]
    df.loc[df["mito_No"] == key, "mito_min_L"] = values["mito_min_L"]
    df.loc[df["mito_No"] == key, "mito_center_L"] = values["mito_center_L"]
    df.loc[df["mito_No"] == key, "lamellar_S"] = values["lamellar_S"]
    df.loc[df["mito_No"] == key, "lamellar_V"] = values["lamellar_V"]
    df.loc[df["mito_No"] == key, "tubular_S"] = values["tubular_S"]
    df.loc[df["mito_No"] == key, "tubular_V"] = values["tubular_V"]
    

In [ ]:
os.makedirs("H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_005", exist_ok = True)

df.to_csv('H:/DeepLearningData/research_010_NIH3T3/shOPA1_003/data/cropped_005/mito_001.csv')